In [1]:
"""
Copyright (c) Meta Platforms, Inc. and affiliates.

This source code is licensed under the CC BY-NC license found in the
LICENSE.md file in the root directory of this source tree.
"""

from torch.utils.tensorboard import SummaryWriter
import argparse
import pickle
import random
import time
import gym

import torch
import numpy as np
import pandas as pd

from datasets import load_from_disk
import datasets

import utils
from replay_buffer import ReplayBuffer
from lamb import Lamb
#from stable_baselines3.common.vec_env import SubprocVecEnv
from pathlib import Path
from data import create_dataloader
from decision_transformer.models.decision_transformer import DecisionTransformer
#from evaluation import create_vec_eval_episodes_fn, vec_evaluate_episode_rtg
from evaluation_citylearn import create_eval_episodes_fn,evaluate_episode_rtg,create_test_episodes_fn
from trainer import SequenceTrainer#, SequenceTrainerCustom
from logger import Logger
from wrappers_custom import *
from utils_.helpers import *

from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import *
from utils_.variant_dict import variant

MAX_EPISODE_LEN = 8760

In [2]:
class self:
    a = 3


In [3]:
self.device = "cuda"

In [4]:
env = CityLearnEnv(schema="citylearn_challenge_2022_phase_1")
env.central_agent = True
env = NormalizedObservationWrapper(env)
env = StableBaselines3WrapperCustom(env)

/home/ml-stud15/miniconda3/envs/stable3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]


In [5]:
dataset_path="data_interactions/model_RBCAgent1_timesteps_8760_rf_CombinedReward_phase_1_new.pkl"

In [6]:
def _get_env_spec(env):
    state_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]
    action_range = [
                -1,#float(env.action_space.low.min()) ,
                1#float(env.action_space.high.max()) ,
            ]
    return state_dim,act_dim, action_range
    
def _get_initial_trajectories(dataset_path):
    dataset = load_from_disk(dataset_path)
    dataset,_ = segment_v2(dataset["observations"],dataset["actions"],dataset["rewards"],dataset["dones"])
    trajectories = datasets.Dataset.from_dict({k: [s[k] for s in dataset] for k in dataset[0].keys()})

    return trajectories

In [7]:
def _load_dataset(trajectories):
    states, traj_lens, returns = [], [], []
    for path in trajectories:
        states.append(path["observations"])
        traj_lens.append(len(path["observations"]))
        returns.append(np.array(path["rewards"]).sum())
    traj_lens, returns = np.array(traj_lens), np.array(returns)

            # used for input normalization
    states = np.concatenate(states, axis=0)
    state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
    num_timesteps = sum(traj_lens)

       
    sorted_inds = np.argsort(returns)  # lowest to highest
    num_trajectories = 1
    timesteps = traj_lens[sorted_inds[-1]]
    ind = len(trajectories) - 2
    while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] <= num_timesteps:
        timesteps += traj_lens[sorted_inds[ind]]
        num_trajectories += 1
        ind -= 1
    sorted_inds = sorted_inds[-num_trajectories:]
    print(sorted_inds)
        #print(trajectories[1])
    for ii in sorted_inds:
        print(ii)
        #print(trajectories[0].keys())
    trajectories = [trajectories[int(ii)] for ii in sorted_inds]

    for trajectory in trajectories:
        for key in trajectory.keys():
            trajectory[key] = np.array(trajectory[key])


    return trajectories, state_mean, state_std

In [8]:
self.state_dim, self.act_dim, self.action_range = _get_env_spec(env)
self.initial_trajectories = _get_initial_trajectories(dataset_path=dataset_path)


/home/ml-stud15/miniconda3/envs/stable3/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.central_agent to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.central_agent` for environment variables or `env.get_wrapper_attr('central_agent')` that will search the reminding wrappers.
  logger.warn(
/home/ml-stud15/miniconda3/envs/stable3/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.buildings to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.buildings` for environment variables or `env.get_wrapper_attr('buildings')` that will search the reminding wrappers.
  logger.warn(
/home/ml-stud15/miniconda3/envs/stable3/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.shared_observations to get variables from other wrappers is deprecated and will be removed in v1.0, to get t

Segmenting:   0%|          | 0/8759 [00:00<?, ?it/s]

In [9]:
self.offline_trajs, self.state_mean, self.state_std = _load_dataset(
            self.initial_trajectories
        )

[0]
0


In [10]:
self.replay_buffer = ReplayBuffer(100000, self.offline_trajs)

In [11]:
self.target_entropy = - self.act_dim

In [12]:
self.hidden_size = 128

In [13]:
self.model = DecisionTransformer(
            state_dim=self.state_dim,
            act_dim=self.act_dim,
            action_range=self.action_range,
            max_length=24,
            eval_context_length=24,
            max_ep_len=8760,
            hidden_size=128,
            n_layer=4,
            n_head=1,
            n_ctx = 72,  # because K = 24
            n_inner=4 * 128,
            activation_function="relu",
            n_positions=1024,
            resid_pdrop=0.1,
            attn_pdrop=0.1,
            stochastic_policy=True,
            ordering=0,
            init_temperature=0.1,
            target_entropy=self.target_entropy
        ).to(device=self.device)

In [14]:
self.model

DecisionTransformer(
  (transformer): GPT2Model(
    (wte): Embedding(1, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (embed_timestep): Embedding(8760, 128)
  (embed_return): Linear(in_features=1, out_features=128, bias=True)
  (embed_state): Linear(in_features=44, out_features=128, bias=True)
  (embed_action): Linear(in_features=5, out_features=128, bias=

In [15]:
dataloader = create_dataloader(
                trajectories=self.replay_buffer.trajectories,
                num_iters=1,
                batch_size=5,
                max_len=24,
                state_dim=self.state_dim,
                act_dim=self.act_dim,
                state_mean=self.state_mean,
                state_std=self.state_std,
                reward_scale=1,
                action_range=self.action_range,
            )

In [16]:
for _, trajs in enumerate(dataloader):
    break

In [17]:
(
            states,
            actions,
            rewards,
            dones,
            rtg,
            timesteps,
            ordering,
            padding_mask,
        ) = trajs

In [18]:
states = states.to(self.device)
actions = actions.to(self.device)
rewards = rewards.to(self.device)
dones = dones.to(self.device)
rtg = rtg.to(self.device)
timesteps = timesteps.to(self.device)
ordering = ordering.to(self.device)
padding_mask = padding_mask.to(self.device)

In [19]:
 action_target = torch.clone(actions)

In [20]:
batch_size, seq_length = states.shape[0], states.shape[1]

In [21]:
state_embeddings = self.model.embed_state(states)
action_embeddings = self.model.embed_action(actions)
returns_embeddings = self.model.embed_return(rtg[:,:-1])

In [22]:
 stacked_inputs = (
            torch.stack(
                (returns_embeddings, state_embeddings, action_embeddings), dim=1
            )
            .permute(0, 2, 1, 3)
            .reshape(batch_size, 3 * seq_length, 128)
        )

In [23]:
stacked_inputs = self.model.embed_ln(stacked_inputs)

In [24]:
stacked_padding_mask = (
            torch.stack((padding_mask, padding_mask, padding_mask), dim=1)
            .permute(0, 2, 1)
            .reshape(batch_size, 3 * seq_length))
        

In [25]:
transformer_outputs = self.model.transformer(
            inputs_embeds=stacked_inputs,
            attention_mask=stacked_padding_mask,
        )

In [26]:
x = transformer_outputs["last_hidden_state"]

In [27]:
x.shape

torch.Size([5, 72, 128])

In [28]:
 x = x.reshape(batch_size, seq_length, 3, self.hidden_size).permute(0, 2, 1, 3)

In [29]:
x.shape

torch.Size([5, 3, 24, 128])

In [30]:
obs = x[:2,1]

In [31]:
mu = self.model.predict_action.mu
log_std = self.model.predict_action.log_std

In [32]:
mu(obs).shape

torch.Size([2, 24, 5])

In [33]:
log_std_value = log_std(obs)

In [34]:
log_std_value =torch.tanh(log_std_value)

In [35]:
log_std_value = -5 + 0.5 * ( 2+5) * (log_std_value + 1.0)

In [36]:
log_std_value

tensor([[[-4.2135, -1.5096,  0.7595,  1.1019, -4.8030],
         [-3.7985, -2.5322,  0.7998,  1.0861, -4.8592],
         [-0.7107, -0.5761, -2.4865,  1.9491, -4.9023],
         [-2.4309,  0.1735, -1.4862,  1.8832, -4.8955],
         [-3.1793, -0.9195, -2.8907,  1.8128, -4.6282],
         [-4.3208, -4.1807,  0.1157,  1.7410, -4.6475],
         [-1.7800, -3.4887, -3.2359,  1.7778, -3.7161],
         [-1.5694, -0.3423, -2.6794,  1.7414, -1.5202],
         [-2.8993, -2.3043, -2.8093,  1.3998,  0.5747],
         [-2.2674,  0.8658, -1.9412,  1.3005,  1.8644],
         [-1.8614,  1.3975,  0.2215,  0.9881,  1.9915],
         [-3.1421,  1.2216,  0.4573,  0.9900,  1.9701],
         [-4.6316,  0.0944, -1.4257,  1.5216,  1.9653],
         [-1.2349, -0.9062, -0.6095, -2.0113,  1.9776],
         [-2.7881, -4.7663, -3.0684, -1.7930,  1.7036],
         [-2.7294, -4.4635,  0.9977, -3.5374,  1.7071],
         [-3.9878, -4.9638, -0.7214, -4.2776,  0.0521],
         [-1.9462, -4.9561, -1.3666, -4.4663, -4

In [37]:
std = log_std_value.exp()

In [38]:
std

tensor([[[1.4794e-02, 2.2099e-01, 2.1372e+00, 3.0099e+00, 8.2052e-03],
         [2.2405e-02, 7.9488e-02, 2.2251e+00, 2.9626e+00, 7.7570e-03],
         [4.9132e-01, 5.6211e-01, 8.3202e-02, 7.0220e+00, 7.4294e-03],
         [8.7953e-02, 1.1895e+00, 2.2624e-01, 6.5744e+00, 7.4803e-03],
         [4.1614e-02, 3.9871e-01, 5.5539e-02, 6.1278e+00, 9.7723e-03],
         [1.3290e-02, 1.5288e-02, 1.1226e+00, 5.7033e+00, 9.5857e-03],
         [1.6864e-01, 3.0541e-02, 3.9324e-02, 5.9170e+00, 2.4330e-02],
         [2.0817e-01, 7.1011e-01, 6.8607e-02, 5.7051e+00, 2.1866e-01],
         [5.5060e-02, 9.9825e-02, 6.0248e-02, 4.0544e+00, 1.7766e+00],
         [1.0358e-01, 2.3769e+00, 1.4353e-01, 3.6712e+00, 6.4518e+00],
         [1.5546e-01, 4.0451e+00, 1.2480e+00, 2.6862e+00, 7.3265e+00],
         [4.3191e-02, 3.3928e+00, 1.5798e+00, 2.6914e+00, 7.1712e+00],
         [9.7391e-03, 1.0990e+00, 2.4033e-01, 4.5795e+00, 7.1369e+00],
         [2.9087e-01, 4.0404e-01, 5.4364e-01, 1.3382e-01, 7.2252e+00],
      

In [39]:
self.loc = mu 

In [40]:
self.std = std

In [41]:
from torch import distributions as pyd
import math
import torch.nn.functional as F
class SquashedNormal(pyd.transformed_distribution.TransformedDistribution):
    """
    Squashed Normal Distribution(s)

    If loc/std is of size (batch_size, sequence length, d),
    this returns batch_size * sequence length * d
    independent squashed univariate normal distributions.
    """

    def __init__(self, loc, std):
        self.loc = loc
        self.std = std

       
        self.base_dist = pyd.Normal(loc, std)

        transforms = [TanhTransform()]
        super().__init__(self.base_dist, transforms)

    @property
    def mean(self):
        mu = self.loc
        for tr in self.transforms:
            mu = tr(mu)
        return mu

    def entropy(self, N=1):
        # sample from the distribution and then compute
        # the empirical entropy:
        x = self.rsample((N,))
        #print(x)
        log_p = self.log_prob(x)
        #print(log_p)
        # log_p: (batch_size, context_len, action_dim),
        return -log_p.mean(axis=0).sum(axis=2)

    def log_likelihood(self, x):
        # log_prob(x): (batch_size, context_len, action_dim)
        # sum up along the action dimensions
        # Return tensor shape: (batch_size, context_len)
        print(self.log_prob(x).sum(axis=2))
        return self.log_prob(x).sum(axis=2)

In [42]:
class TanhTransform(pyd.transforms.Transform):
    domain = pyd.constraints.real
    codomain = pyd.constraints.interval(-1.0, 1.0)
    bijective = True
    sign = +1

    def __init__(self, cache_size=1):
        super().__init__(cache_size=cache_size)

    @staticmethod
    def atanh(x):
        return 0.5 * (x.log1p() - (-x).log1p())

    def __eq__(self, other):
        return isinstance(other, TanhTransform)

    def _call(self, x):
        return x.tanh()

    def _inverse(self, y):
        # We do not clamp to the boundary here as it may degrade the performance of certain algorithms.
        # one should use `cache_size=1` instead
        return self.atanh(y)

    def log_abs_det_jacobian(self, x, y):
        # We use a formula that is more numerically stable, see details in the following link
        # https://github.com/tensorflow/probability/commit/ef6bb176e0ebd1cf6e25c6b5cecdd2428c22963f#diff-e120f70e92e6741bca649f04fcd907b7
        return 2.0 * (math.log(2.0) - x - F.softplus(-2.0 * x))

In [43]:
sn = SquashedNormal(mu(obs), std)

In [288]:
x = sn.rsample((1,))


In [289]:
print(x[0][0][0])

tensor([ 0.3696, -0.5529, -0.4020,  0.5891,  0.9422], device='cuda:0',
       grad_fn=<SelectBackward0>)


In [290]:
y = x.clone()

In [291]:
y[0][0][0] = 0.38

In [292]:
x

tensor([[[[ 0.3696, -0.5529, -0.4020,  0.5891,  0.9422],
          [ 0.4523, -0.3470,  0.9783, -0.9387,  0.9063],
          [ 0.9266,  0.7319, -0.4236, -1.0000,  0.8482],
          [ 0.9697,  0.9960, -0.7295,  0.9943,  0.9371],
          [ 0.9746,  0.6204, -0.7538,  1.0000,  0.6587],
          [ 0.7566,  0.7022, -0.5542,  0.9875,  0.7242],
          [ 0.9698,  0.7633, -0.7236, -0.7452,  0.7376],
          [ 0.7890,  0.6245, -0.7072,  0.5435,  0.7598],
          [ 0.7638,  0.9291, -0.9154,  0.9243,  0.8878],
          [ 0.7102, -0.9881, -0.6016,  0.9994,  1.0000],
          [ 0.4932, -0.9985, -0.9041, -0.0243, -1.0000],
          [ 0.1368,  0.0087, -0.3986,  0.9746, -1.0000],
          [ 0.2831, -0.9772, -0.7961, -0.9927,  0.9879],
          [-0.4609, -0.2340, -0.8441,  0.6786, -1.0000],
          [-0.7104,  0.4236, -0.8782,  0.8807, -0.9971],
          [-0.9444,  0.6906,  0.9201,  0.5492, -1.0000],
          [-0.9473,  0.9313, -0.7825,  0.8693, -0.8489],
          [-0.8486,  0.6119, -0

In [293]:
sn.base_dist

Normal(loc: torch.Size([2, 24, 5]), scale: torch.Size([2, 24, 5]))

In [294]:
log_p = sn.log_prob(y)

In [295]:
log_p[0][0]

tensor([[ 2.6376e+00, -9.2048e+00, -1.5225e+00, -2.0680e+00, -1.3627e+04],
        [ 3.1035e+00,  1.2642e+00,  1.3576e+00,  8.7902e-02,  4.2187e+00],
        [ 1.3502e+00,  2.8447e-01,  1.7630e+00,         nan,  4.9361e+00],
        [ 4.2751e+00,  1.4729e+00,  9.5949e-01,  1.6012e+00,  5.4260e+00],
        [ 4.3864e+00,  2.7942e-01,  1.8982e+00,         nan,  3.7978e+00],
        [ 4.0901e+00,  3.5447e+00, -9.9325e-01,  1.0112e+00,  4.0922e+00],
        [ 3.2416e+00,  3.3114e+00,  1.2673e+00, -1.9263e+00,  1.9340e+00],
        [ 1.6189e+00, -1.3587e-01,  2.4493e+00, -2.3107e+00,  1.2027e+00],
        [ 2.4819e+00,  3.3743e+00,  3.0600e+00, -4.1837e-01,  5.1227e-02],
        [ 2.0409e+00,  1.1078e+00,  1.4024e+00,  3.8307e+00,  6.7176e+00],
        [ 1.1630e+00,  2.7945e+00,  5.4040e-01, -1.9104e+00,         nan],
        [ 2.1443e+00, -2.1978e+00, -1.2182e+00,  7.9319e-01,  6.6761e+00],
        [ 3.6162e+00, -9.0543e-01,  1.4278e+00,  1.4417e+00,  8.3017e-01],
        [-9.0522e-01, -1.

In [254]:
sn.std[0]

tensor([[1.4794e-02, 2.2099e-01, 2.1372e+00, 3.0099e+00, 8.2052e-03],
        [2.2405e-02, 7.9488e-02, 2.2251e+00, 2.9626e+00, 7.7570e-03],
        [4.9132e-01, 5.6211e-01, 8.3202e-02, 7.0220e+00, 7.4294e-03],
        [8.7953e-02, 1.1895e+00, 2.2624e-01, 6.5744e+00, 7.4803e-03],
        [4.1614e-02, 3.9871e-01, 5.5539e-02, 6.1278e+00, 9.7723e-03],
        [1.3290e-02, 1.5288e-02, 1.1226e+00, 5.7033e+00, 9.5857e-03],
        [1.6864e-01, 3.0541e-02, 3.9324e-02, 5.9170e+00, 2.4330e-02],
        [2.0817e-01, 7.1011e-01, 6.8607e-02, 5.7051e+00, 2.1866e-01],
        [5.5060e-02, 9.9825e-02, 6.0248e-02, 4.0544e+00, 1.7766e+00],
        [1.0358e-01, 2.3769e+00, 1.4353e-01, 3.6712e+00, 6.4518e+00],
        [1.5546e-01, 4.0451e+00, 1.2480e+00, 2.6862e+00, 7.3265e+00],
        [4.3191e-02, 3.3928e+00, 1.5798e+00, 2.6914e+00, 7.1712e+00],
        [9.7391e-03, 1.0990e+00, 2.4033e-01, 4.5795e+00, 7.1369e+00],
        [2.9087e-01, 4.0404e-01, 5.4364e-01, 1.3382e-01, 7.2252e+00],
        [6.1535e-02,

In [255]:
sn.loc[0]

tensor([[ 3.8120e-01, -5.8586e-01,  4.2618e-01, -1.5184e+00,  1.7548e+00],
        [ 4.8975e-01, -2.8436e-01,  1.4034e+00, -9.2663e-01,  1.5196e+00],
        [ 1.1956e+00,  6.3560e-01, -4.5772e-01, -2.7310e-02,  1.2556e+00],
        [ 2.0582e+00,  5.7448e-01, -7.3370e-01,  4.4102e-01,  1.7048e+00],
        [ 2.1222e+00,  4.6903e-01, -9.0659e-01,  5.8933e-01,  7.8085e-01],
        [ 9.8057e-01,  8.5809e-01, -1.5303e+00,  1.2922e+00,  9.2478e-01],
        [ 1.9303e+00,  9.8843e-01, -9.8956e-01,  7.0813e-01,  9.0102e-01],
        [ 1.0463e+00,  9.6478e-01, -8.7501e-01,  7.8048e-01,  1.1533e+00],
        [ 9.5770e-01,  1.6484e+00, -1.6288e+00,  6.8575e-01,  1.6203e+00],
        [ 9.0173e-01,  5.3937e-01, -7.4908e-01, -1.2979e-01,  1.9438e+00],
        [ 5.9318e-01,  1.2531e+00, -1.2469e+00,  2.1368e-01,  1.8525e+00],
        [ 1.5679e-01,  1.1568e+00, -6.9561e-01,  1.2996e-01,  1.5173e+00],
        [ 2.8517e-01,  4.5681e-01, -1.1861e+00,  1.0283e+00,  1.4310e+00],
        [-1.4684e-03,  2.

In [264]:
log_p = sn.log_prob(x)

In [265]:
final = -log_p.mean(axis=0).sum(axis=2)

In [266]:
final.mean()

tensor(-15.2336, device='cuda:0', grad_fn=<MeanBackward0>)

In [165]:
sn.std

tensor([[[3.4046e+00, 5.9851e-02, 5.0004e+00, 2.4783e-01, 4.1118e-02],
         [1.6766e+00, 5.1901e-02, 1.1728e+00, 3.4660e+00, 4.1634e-01],
         [2.2673e+00, 1.0006e+00, 1.8186e-01, 7.2136e-01, 1.9689e-01],
         [5.7206e-01, 3.4250e-01, 4.7609e-01, 3.0630e+00, 2.3964e-02],
         [4.5346e-01, 1.2693e+00, 7.7443e-02, 5.1055e-01, 1.8801e-02],
         [3.1743e-01, 2.9955e+00, 3.9022e-01, 2.4050e+00, 9.2742e-02],
         [5.4768e-01, 1.2794e+00, 3.3353e-01, 1.3763e+00, 3.2633e-02],
         [7.3193e-03, 4.6911e+00, 8.8040e-01, 6.2376e-01, 6.9694e-02],
         [6.8511e-03, 5.9260e+00, 2.3426e-02, 7.8259e-02, 8.2174e-01],
         [6.8398e-03, 6.1229e+00, 1.0607e-02, 3.3317e-01, 3.1649e+00],
         [6.9498e-03, 6.8386e+00, 2.0350e-02, 2.5917e-02, 1.9217e+00],
         [7.1681e-03, 4.1406e+00, 6.8452e-01, 6.9836e-01, 6.2630e-01],
         [6.9975e-03, 3.2531e+00, 6.6677e-02, 3.1164e-02, 6.1267e-02],
         [7.3086e-03, 2.6281e+00, 3.2420e-01, 1.1694e-02, 2.4513e-02],
      

In [302]:
value = x
self.loc = sn.loc
log_scale = sn.std.log()
var = sn.std ** 2
test = -((value - self.loc) ** 2) / (2 * var)  - log_scale - math.log(math.sqrt(2 * math.pi))

In [312]:
v_s = value[0][0][0][0]

In [313]:
loc_s = self.loc[0][0][0]

In [314]:
var_s = var[0][0][0]

In [317]:
l_s = log_scale[0][0][0]

In [319]:
log_scale

tensor([[[-4.2135, -1.5096,  0.7595,  1.1019, -4.8030],
         [-3.7985, -2.5322,  0.7998,  1.0861, -4.8592],
         [-0.7107, -0.5761, -2.4865,  1.9491, -4.9023],
         [-2.4309,  0.1735, -1.4862,  1.8832, -4.8955],
         [-3.1793, -0.9195, -2.8907,  1.8128, -4.6282],
         [-4.3208, -4.1807,  0.1157,  1.7410, -4.6475],
         [-1.7800, -3.4887, -3.2359,  1.7778, -3.7161],
         [-1.5694, -0.3423, -2.6794,  1.7414, -1.5202],
         [-2.8993, -2.3043, -2.8093,  1.3998,  0.5747],
         [-2.2674,  0.8658, -1.9412,  1.3005,  1.8644],
         [-1.8614,  1.3975,  0.2215,  0.9881,  1.9915],
         [-3.1421,  1.2216,  0.4573,  0.9900,  1.9701],
         [-4.6316,  0.0944, -1.4257,  1.5216,  1.9653],
         [-1.2349, -0.9062, -0.6095, -2.0113,  1.9776],
         [-2.7881, -4.7663, -3.0684, -1.7930,  1.7036],
         [-2.7294, -4.4635,  0.9977, -3.5374,  1.7071],
         [-3.9878, -4.9638, -0.7214, -4.2776,  0.0521],
         [-1.9462, -4.9561, -1.3666, -4.4663, -4

In [322]:
sn.std

tensor([[[1.4794e-02, 2.2099e-01, 2.1372e+00, 3.0099e+00, 8.2052e-03],
         [2.2405e-02, 7.9488e-02, 2.2251e+00, 2.9626e+00, 7.7570e-03],
         [4.9132e-01, 5.6211e-01, 8.3202e-02, 7.0220e+00, 7.4294e-03],
         [8.7953e-02, 1.1895e+00, 2.2624e-01, 6.5744e+00, 7.4803e-03],
         [4.1614e-02, 3.9871e-01, 5.5539e-02, 6.1278e+00, 9.7723e-03],
         [1.3290e-02, 1.5288e-02, 1.1226e+00, 5.7033e+00, 9.5857e-03],
         [1.6864e-01, 3.0541e-02, 3.9324e-02, 5.9170e+00, 2.4330e-02],
         [2.0817e-01, 7.1011e-01, 6.8607e-02, 5.7051e+00, 2.1866e-01],
         [5.5060e-02, 9.9825e-02, 6.0248e-02, 4.0544e+00, 1.7766e+00],
         [1.0358e-01, 2.3769e+00, 1.4353e-01, 3.6712e+00, 6.4518e+00],
         [1.5546e-01, 4.0451e+00, 1.2480e+00, 2.6862e+00, 7.3265e+00],
         [4.3191e-02, 3.3928e+00, 1.5798e+00, 2.6914e+00, 7.1712e+00],
         [9.7391e-03, 1.0990e+00, 2.4033e-01, 4.5795e+00, 7.1369e+00],
         [2.9087e-01, 4.0404e-01, 5.4364e-01, 1.3382e-01, 7.2252e+00],
      